# Lesson 3: Preparing Text Data for RAG

<p padding:15px; margin-left:20px"> <b>Note:</b> RAG systems start by using vector representations of text to match your prompt to relevant sections within the unstructured data. So in order to be able to find relevant text in a knowledge graph in the same way, need to creat embedding in the graph</p>


### Import packages and set up Neo4j

In [1]:
from dotenv import load_dotenv
import os

from langchain_community.graphs import Neo4jGraph

# Warning control
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load from environment
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Note the code below is unique to this course environment, and not a 
# standard part of Neo4j's integration with OpenAI. Remove if running 
# in your own environment.
# open_api_key to call the openAPI embeding model
OPENAI_ENDPOINT = os.getenv('OPENAI_BASE_URL') + '/embeddings'

In [3]:
# Connect to the knowledge graph instance using LangChain
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

# Vector Search:

### Create a vector index 

In [4]:
# dimensions (default 1536) and similarity_function is attributes for openAPI model
kg.query("""
  CREATE VECTOR INDEX movie_tagline_embeddings IF NOT EXISTS
  FOR (m:Movie) ON (m.taglineEmbedding) 
  OPTIONS { indexConfig: {
    `vector.dimensions`: 1536,
    `vector.similarity_function`: 'cosine'
  }}"""
)


[]

In [5]:
kg.query("""
  SHOW VECTOR INDEXES
  """
)

[{'id': 3,
  'name': 'movie_tagline_embeddings',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'VECTOR',
  'entityType': 'NODE',
  'labelsOrTypes': ['Movie'],
  'properties': ['taglineEmbedding'],
  'indexProvider': 'vector-2.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': 0}]

### Populate the vector index
- Calculate vector representation for each movie tagline using OpenAI
- Add vector to the `Movie` node as `taglineEmbedding` property
- Requre GenAI plugin, https://neo4j.com/docs/cypher-manual/current/genai-integrations/
- Search for 'step by step of how to install the GenAI plugin Neo4j with community edition on local machine'

In [6]:
# genai.vector.encode(value_to_encode, embeding_model_to_use)
# further details on this function  https://neo4j.com/docs/cypher-manual/current/genai-integrations/
# $ query parameter
kg.query("""
    MATCH (movie:Movie) WHERE movie.tagline IS NOT NULL
    WITH movie, genai.vector.encode(
        movie.tagline, 
        "OpenAI", 
        {
          token: $openAiApiKey,
          endpoint: $openAiEndpoint
        }) AS vector
    CALL db.create.setNodeVectorProperty(movie, "taglineEmbedding", vector)
    """, 
    params={"openAiApiKey":OPENAI_API_KEY, "openAiEndpoint": OPENAI_ENDPOINT} )

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke function `genai.vector.encode`: Caused by: org.neo4j.genai.vector.MalformedGenAIResponseException: Unexpected HTTP response code: 429 Too Many Requests - {
    "error": {
        "message": "You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.",
        "type": "insufficient_quota",
        "param": null,
        "code": "insufficient_quota"
    }
}
}

In [7]:
result = kg.query("""
    MATCH (m:Movie) 
    WHERE m.tagline IS NOT NULL
    RETURN m.tagline, m.taglineEmbedding
    LIMIT 1
    """
)

In [8]:
result[0]['m.tagline']

'Welcome to the Real World'

In [9]:
result[0]['m.taglineEmbedding'][:10]

TypeError: 'NoneType' object is not subscriptable

In [ ]:
len(result[0]['m.taglineEmbedding'])

### Similarity search
- Calculate embedding for question
- Identify matching movies based on similarity of question and `taglineEmbedding` vectors

In [ ]:
question = "What movies are about love?"

In [ ]:
# db.index.vector.queryNodes https://neo4j.com/docs/operations-manual/5/reference/procedures/#procedure_db_index_vector_queryNodes
# calculate the embeding for the query, db.index.vector.queryNodes (similarity search) to search vector index and return nearest neighbour to query vector

kg.query("""
    WITH genai.vector.encode(
        $question, 
        "OpenAI", 
        {
          token: $openAiApiKey,
          endpoint: $openAiEndpoint
        }) AS question_embedding
    CALL db.index.vector.queryNodes(
        'movie_tagline_embeddings', 
        $top_k, 
        question_embedding
        ) YIELD node AS movie, score
    RETURN movie.title, movie.tagline, score
    """, 
    params={"openAiApiKey":OPENAI_API_KEY,
            "openAiEndpoint": OPENAI_ENDPOINT,
            "question": question,
            "top_k": 5
            })

### Try for yourself: ask you own question!
- Change the question below and run the graph query to find different movies

In [ ]:
question = "What movies are about adventure?"

In [ ]:
kg.query("""
    WITH genai.vector.encode(
        $question, 
        "OpenAI", 
        {
          token: $openAiApiKey,
          endpoint: $openAiEndpoint
        }) AS question_embedding
    CALL db.index.vector.queryNodes(
        'movie_tagline_embeddings', 
        $top_k, 
        question_embedding
        ) YIELD node AS movie, score
    RETURN movie.title, movie.tagline, score
    """, 
    params={"openAiApiKey":OPENAI_API_KEY,
            "openAiEndpoint": OPENAI_ENDPOINT,
            "question": question,
            "top_k": 5
            })